In [1]:
import pandas as pd
import numpy as np
import json
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from osgeo import ogr
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import datetime
import re
import jieba
import mysql.connector
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy import create_engine
import model
import time
import googlemaps
import os

In [2]:
Session = sessionmaker(bind=model.engine)

In [20]:
#model.drop_db()
#model.init_db()

In [1]:
#### define all types in google data ,remove thoat not in this types
foodtypes=['餐廳', '餐館', '咖啡', '熟食', '冰品', '飲料', '餃子', '牛扒', '果汁', 
      '早餐', '酒吧',  '拉麵', '居酒屋', '酒店', '雪糕', '甜品', '麵店', '麵包', 
      '冷麵', '麵食店', '美式扒房','熟食店', '珍珠奶茶', '包點', '壽司', 
      '烏冬專門店', '菜館', '三文治店', '薄餅', '曲奇餅店',  '酒樓', '海味店', 
      '餡餅店', '快餐', '酒家', '飲料店', '茶館', '餅店', '健康食品專賣店','燒烤', 
      '小食店','餐車店','餐廳','茶藝','麵館','冬甩','美食廣場','烤雞','速食','班戟',
      '擔擔麵']
#fstoretypes=['便利店','市場']
nottypes=['用品店','批發商','農業用品店','供應商','溫室','住宿','肉販','批發','農產品','工作空間','書店']

In [22]:
###   檢查是否存在上述的foodtypes， 如果存在於nottypes 將會去除  避免ㄧ些怪怪的
def checkFood(onetype,types):
    boFood=False
    for t in types:
        if(t in onetype):
            boFood=True
            break
    return boFood
def checkNotFood(onetype,types):
    boFood=False
    for t in types:
        if(t in onetype):
            boFood=True
            break
    return boFood   

##### 滿足 googletype  為上述的成分
#### 滿足 types 裡面有 food 的因子o
def get_check(data):
    boCheck=[]
    for i,j in zip(data['googletype'],data['type']):
        boTmp=False
        if('food' in str(j)):
            boTmp=True
        if( checkFood(str(i),foodtypes) ):
            boTmp=True
        if(checkNotFood(str(i),nottypes)):
            boTmp=False
        boCheck.append(boTmp)
    return boCheck

In [24]:
def get_merge_google(path,inputName):
    dataGoogleApi=pd.read_csv(path+'GoogleApi/'+inputName+'.csv')
    dataGoogleInfo=pd.read_csv(path+'GoogleInfo/'+inputName+'.csv')
    dataGoogle=pd.merge(dataGoogleApi,dataGoogleInfo,on='placeid')
    if os.path.isdir(path+'merage/'):
        pass
    else:
        os.makedirs(path+'merage/')
    dataGoogle.to_csv(path+'merage/'+inputName+'.csv')
    dataGoogle.columns=['from', 'icon', 'id', 'lat', 'lon', 'name', 'placeid', 'price',
       'rating', 'type', 'user_ratings_total', 'vicinity', 'url',
       'googletype', 'imageUrl', 'opentime', 'phone', 'ray', 'review',
       'webUrl']
    return dataGoogle

In [25]:
### 定義一下高雄區域，利用地址分類資料  ######
def get_define_place(data):
    kao=['鼓山區','三民區','新興區','前金區','苓雅區','鹽埕區','前鎮區','旗津區','左營區','鳳山區','大寮區','大樹區','鳥松區','小港區','楠梓區','林園區','岡山區','仁武區','大社區','旗山區']
    place='高雄'
    area=[]
    city=[]
    for st,i in enumerate(data['vicinity']):
        areatmp=None
        placetmp=None
        for j in kao:
            if('鳳山市' in i):
                areatmp='鳳山區'
                placetmp=place            
            if(j in i):
                areatmp=j
                placetmp=place
        if(pd.isnull(areatmp)):print(i)
        area.append(areatmp)
        city.append(placetmp)
    data['area']=area
    data['city']=city
    return data

In [43]:
#將資料儲存到DB裡面
def insert_DB(food):
    session = Session()
    food=food.where(food.notnull(), None)
    
    for st in range(len(food)):
        switch_command = session.query(model.google).\
                            filter(model.google.placeid == food['placeid'][st]).first()
        #檢查資料庫有資料
        if switch_command:
        #更新資料
            pass
        else:
        #建立資料
            #food['rating'][st]='NULL' if pd.isna(food['rating'][st])else food['rating'][st]
            #food['rating'][st]='NULL' if pd.isna(food['rating'][st])else food['user_ratings_total'][st]
#             food = food.fillna('NULL')
            food['name'][st]=str(food['name'][st]).replace('|','').replace('、','').replace(',','').replace(':','')
            food['type'][st]=str(food['type'][st]).replace(',','_')
            food['vicinity'][st]=str(food['vicinity'][st]).replace(',','')
        
            session.add(model.google(icon=food['icon'][st],\
                             lat=food['lat'][st],\
                             lon=food['lon'][st],\
                             name=food['name'][st],\
                             placeid=food['placeid'][st],\
                             rating=food['rating'][st],\
                             review=food['user_ratings_total'][st],\
                             price=food['price'][st],\
                             url=food['url'][st],\
                             types=food['type'][st],\
                             phone=food['phone'][st],\
                             vicinity=food['vicinity'][st],\
                             googletype=food['googletype'][st],\
                             imageUrl=food['imageUrl'][st],\
                             webUrl=food['webUrl'][st],\
                             area=food['area'][st],\
                             city=food['city'][st],\
                             opentime=food['opentime'][st]))
            session.commit()
    session.close()

In [44]:
if __name__ == '__main__':
    ### 讀取資料!!!
    inputName='Xinxing'
    path='input/Kaohsiung/'
    dataGoogle=get_merge_google(path,inputName)
    #### 取出  food 的內容
    fo
    #### 增加地理念 哪一區
    food=get_define_place(food)
    insert_DB(food)

中正四路71號
高雄市
南台路128號
No. 163-1, Linsen 1st Road, Xinxing District
3F, No, No. 267, 林森一路


In [1]:
import googledb

In [2]:
inputName='Xinxing'
path='input/Kaohsiung/'

In [9]:
data=googledb.GoogleDB(inputName,path).merage()

In [4]:
#### define all types in google data ,remove thoat not in this types
foodtypes=['餐廳', '餐館', '咖啡', '熟食', '冰品', '飲料', '餃子', '牛扒', '果汁', 
      '早餐', '酒吧',  '拉麵', '居酒屋', '酒店', '雪糕', '甜品', '麵店', '麵包', 
      '冷麵', '麵食店', '美式扒房','熟食店', '珍珠奶茶', '包點', '壽司', 
      '烏冬專門店', '菜館', '三文治店', '薄餅', '曲奇餅店',  '酒樓', '海味店', 
      '餡餅店', '快餐', '酒家', '飲料店', '茶館', '餅店', '健康食品專賣店','燒烤', 
      '小食店','餐車店','餐廳','茶藝','麵館','冬甩','美食廣場','烤雞','速食','班戟',
      '擔擔麵']
nottypes=['用品店','批發商','農業用品店','供應商','溫室','住宿','肉販','批發','農產品',\
          '工作空間','書店','公關公司','食品商店','食品製造供應','珠寶商','唱片店','魚販',\
          '網吧','餐具店','禮盒專賣店','出版社','到會','藥房']

In [5]:
check=googledb.CheckFood(foodtypes,nottypes,data).get_check()

In [13]:
data['type'][0]

"['locality', 'administrative_area_level_1', 'political']"

In [ ]:
if 'food' in i for i in data['type']

In [15]:
bo=[]
for i in data['type']:
    if('food' in i):
        bo.append(True)
    else:
        bo.append(False)

In [8]:
check.to_csv('test.csv',encoding='utf_8_sig')

In [2]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
list

list

In [5]:
msg = input(">> ")
if "悲傷" in msg:
    print("拍拍")
else:
    print("got it")

>>  dd


got it


In [7]:
%pylab inline
randint(3,21)

Populating the interactive namespace from numpy and matplotlib


10

In [14]:
import random

In [16]:
random.randint(3,21)

5

In [8]:
for i in range(10):
    k = randint(1,50)
    print(" "*k+"*")

                                              *
                                          *
         *
                 *
              *
                             *
                                  *
                              *
   *
                                       *
